In [1]:
import h5py
import trimesh
import os
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def load_hdf5(file_path):
    with h5py.File(file_path, 'r') as f:
        data = {key: f[key][()] for key in f.keys()}
    return data


In [ ]:
# 合成一个物体
# 使用现成的chair ——>打上part label

def generate_pth_cls(id,part_id):
    mesh_base_dir = '/home/lidosan/Datasets/datasets--ShapeNet--PartNet-archive/blobs/data_v0/{}/objs/new-{}.obj'.format(id,part_id)
    return mesh_base_dir

In [ ]:
import os
import json
import trimesh
import numpy as np

# Load JSON file
def load_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Print hierarchical structure
def print_structure(data, level=0):
    indent = '  ' * level
    if isinstance(data, dict):
        if 'text' in data:
            print(f"{indent}{data['text']} (id: {data.get('id', 'N/A')}, name: {data.get('name', 'N/A')})")
        if 'objs' in data:
            print(f"{indent}  Mesh Files: {data['objs']}")
        if 'children' in data:
            for child in data['children']:
                print_structure(child, level + 1)
    elif isinstance(data, list):
        for item in data:
            print_structure(item, level)

# Extract part labels and their corresponding mesh files
# Extract part labels and their corresponding mesh files
def extract_parts(data, parent_label=''):
    parts = []
    if isinstance(data, dict):
        if 'text' in data:
            label = parent_label + ' -> ' + data['text'] if parent_label else data['text']
            if 'objs' in data:
                parts.append((label, data['objs']))
            if 'children' in data:
                for child in data['children']:
                    parts.extend(extract_parts(child, label))
    elif isinstance(data, list):
        for item in data:
            parts.extend(extract_parts(item, parent_label))
    return parts

# Generate a color palette for labels
def generate_color_palette(num_colors):
    return np.random.rand(num_colors, 3)


# Load and color meshes
def load_and_color_meshes_instance(obj_id, parts):
    obj_base_dir = f'/home/lidosan/Datasets/datasets--ShapeNet--PartNet-archive/blobs/data_v0/{obj_id}/objs'
    combined_mesh = trimesh.Scene()

    # Flatten the list of parts to get all unique object IDs
    all_object_ids = [obj for _, objs in parts for obj in objs]
    color_palette = generate_color_palette(len(all_object_ids))
    object_id_to_color = {obj_id: (color_palette[i] * 255).astype(np.uint8) for i, obj_id in enumerate(all_object_ids)}

    for label, objs in parts:
        for obj in objs:
            color = object_id_to_color[obj]
            mesh_path = os.path.join(obj_base_dir, f"{obj}.obj")
            if os.path.exists(mesh_path):
                mesh = trimesh.load(mesh_path)
                # Assign a color to the mesh
                mesh.visual.vertex_colors = np.tile(color, (len(mesh.vertices), 1))
                # Add mesh to the scene
                combined_mesh.add_geometry(mesh)
            else:
                print(f"Mesh file {mesh_path} not found!")

    return combined_mesh



# Load and color meshes by label
def load_and_color_meshes_semantics(obj_id, parts):
    obj_base_dir = f'/home/lidosan/Datasets/datasets--ShapeNet--PartNet-archive/blobs/data_v0/{obj_id}/objs'
    combined_mesh = trimesh.Scene()

    unique_labels = list(set(label for label, _ in parts))
    color_palette = generate_color_palette(len(unique_labels))
    label_to_color = {label: color_palette[i] for i, label in enumerate(unique_labels)}

    print(label_to_color)
    for label, objs in parts:
        color = label_to_color[label]
        for obj in objs:
            mesh_path = os.path.join(obj_base_dir, f"{obj}.obj")
            if os.path.exists(mesh_path):
                mesh = trimesh.load(mesh_path)
                # Assign the color to the mesh
                mesh.visual.vertex_colors = np.tile(color, (mesh.vertices.shape[0], 1))
                # Add mesh to the scene
                combined_mesh.add_geometry(mesh)
            else:
                print(f"Mesh file {mesh_path} not found!")

    return combined_mesh



obj_id = '15052'

obj_base_dir = '/home/lidosan/Datasets/datasets--ShapeNet--PartNet-archive/blobs/data_v0/{}'.format(obj_id)

json_file_path = 'chair_structure.json'
chair_structure = load_json(os.path.join(obj_base_dir,'result.json'))

print("Hierarchical Structure of Chair and Its Parts:")
print_structure(chair_structure)

# Extract parts with labels and object files
parts = extract_parts(chair_structure)
print("Extracted Parts and Corresponding Mesh Files:")
for label, objs in parts:
    print(f"{label}: {objs}")

# Example object ID

# Load, color, and visualize the combined mesh
combined_mesh = load_and_color_meshes_semantics(obj_id, parts)
if combined_mesh:
    print("Combined mesh with colored parts successfully created.")
    combined_mesh.show()
else:
    print("No meshes were loaded.")


In [ ]:
combined_mesh.show()
combined_mesh.export('scene1/lamp_{}.obj'.format(obj_id))


In [ ]:
table_list = os.listdir('/home/lidosan/Datasets/PartNet/Table/models')

In [ ]:
id = '15052.obj'
category = 'Lamp'
path = '/home/lidosan/Datasets/PartNet/{}/models/{}'.format(category,id)

In [ ]:
trimesh.load(path).show()